# Python을 활용한 web scraping 기초

2021.08.12

---

# Introduction

* 초보자 대상
* 기본 구조를 이해하고 활용하기 위한 기초 단계 
* 가장 기본적인 방법을 통해 데이터 가져오기

# Responsible web scraping

* Terms of Service, robots.txt
* Open API
* Adequate request rate

# 예제 2: Job-Alio 공공기관정보

공공기관정보: https://job.alio.go.kr/orginfo.do

In [ ]:
from selenium import webdriver
import pandas as pd
import time

driver = webdriver.Chrome('./chromedriver')

In [ ]:
url = 'https://job.alio.go.kr/orginfo.do'
driver.get(url)

## 브라우저 제어

### Checkbox

In [ ]:
checkbox1 = driver.find_element_by_xpath('//*[@id="so01"]')
checkbox1.click()

### Dropdown

In [ ]:
from selenium.webdriver.support.ui import Select
dropdown1 = Select(driver.find_element_by_xpath('//*[@id="jumu01"]'))
dropdown1.select_by_value('A1003')

### Search & Reset

In [ ]:
driver.execute_script('javascript:fn_search();')

In [ ]:
driver.execute_script('javascript:fn_reset();')

## 리스트 수집 - 방식 1

In [1]:
from selenium import webdriver
import pandas as pd
import time

driver = webdriver.Chrome('./chromedriver')
url = 'https://job.alio.go.kr/orginfo.do'
driver.get(url)

In [2]:
page_number = 1
more_page = True

pub_name = []
pub_gov = []
pub_type = []
pub_homepage = []
pub_url = []

while more_page:
    driver.execute_script(f'goPage({page_number});return false;')
    time.sleep(2)
    print(f'processing page {page_number}')
    
    rows = driver.find_elements_by_xpath('//*[@id="txt"]/table/tbody/tr')
    
    for r in rows:
        pub_name.append(r.find_element_by_xpath('td[2]').text)
        pub_gov.append(r.find_element_by_xpath('td[3]').text)
        pub_type.append(r.find_element_by_xpath('td[4]').text)
        pub_homepage.append(r.find_element_by_xpath('td[5]').text)
        pub_url.append(r.find_element_by_xpath('td[2]/a').get_attribute('onclick'))
    
    page_number += 1
    if page_number == 2:
        more_page = False

processing page 1


In [3]:
pub_df = pd.DataFrame(list(zip(pub_name, pub_gov, pub_type, pub_homepage, pub_url)), 
                      columns=['name', 'gov', 'type', 'homepage', 'url'])

In [4]:
pub_df

,name,gov,type,homepage,url
0,(사)남북교류협력지원협회,통일부,기타공공기관,http://www.sonosa.or.kr,javascript:fnDetail('C0847');
1,(재)APEC기후센터,기상청,기타공공기관,http://www.apcc21.org,javascript:fnDetail('C0918');
2,(재)국제원산지정보원,관세청,기타공공기관,http://www.origin.or.kr,javascript:fnDetail('C0916');
3,(재)예술경영지원센터,문화체육관광부,기타공공기관,http://www.gokams.or.kr,javascript:fnDetail('C0454');
4,(재)우체국금융개발원,과학기술정보통신부,준정부기관(위탁집행형),http://www.posid.or.kr,javascript:fnDetail('C0352');
5,(재)우체국물류지원단,과학기술정보통신부,준정부기관(위탁집행형),http://www.pola.or.kr,javascript:fnDetail('C0331');
6,(재)우체국시설관리단,과학기술정보통신부,기타공공기관,http://www.poma.or.kr,javascript:fnDetail('C0368');
7,(재)일제강제동원피해자지원재단,행정안전부,기타공공기관,https://www.ilje.or.kr/,javascript:fnDetail('C0931');
8,(재)축산환경관리원,농림축산식품부,기타공공기관,http://www.lemi.or.kr/,javascript:fnDetail('C1030');
9,(재)한국우편사업진흥원,과학기술정보통신부,준정부기관(위탁집행형),http://www.posa.or.kr,javascript:fnDetail('C0354');


## 리스트 수집 - 방식 2

In [ ]:
page_number = 1
pub_list = []

while True:
    driver.execute_script(f'goPage({page_number});return false;')
    time.sleep(2)
    print(f'processing page {page_number}')
    
    table = driver.find_element_by_css_selector('#txt > table > tbody')
    rows = table.find_elements_by_tag_name('tr')
    
    for r in rows:
        pub_list.append({
            'name':r.find_elements_by_css_selector('td')[1].text,
            'gov':r.find_elements_by_css_selector('td')[2].text,
            'type':r.find_elements_by_css_selector('td')[3].text,
            'homepage':r.find_elements_by_css_selector('td')[4].text,
            'url':r.find_element_by_css_selector('td.left > a').get_attribute('onclick')
        })
    page_number += 1
    if page_number == 2:
        break

In [ ]:
pub_df = pd.DataFrame(pub_list)

In [ ]:
pub_df

## 본문 수집 - 방식 1

In [ ]:
pub_contents = []

for u in pub_df['url']:
    driver.execute_script(u)
    time.sleep(2)
    
    table = driver.find_element_by_css_selector('#txt > table')
    headers = table.find_elements_by_tag_name('th')
    contents = table.find_elements_by_tag_name('td')
    header_list = []
    content_list = []
    
    for h in range(len(headers)):
        header_list.append(headers[h].text)
        content_list.append(contents[h].text)
        
    alio_dict = {}
    
    for n in range(len(header_list)):
        alio_dict[header_list[n]] = content_list[n]
        
    pub_contents.append(alio_dict)
    
    driver.back()
    time.sleep(2)

In [ ]:
pub_content_df = pd.DataFrame(pub_contents)
pub_content_df

In [ ]:
pub_df_merge = pd.merge(pub_df, pub_content_df, left_on='name', right_on='기관명', how='left')

In [ ]:
pub_df_merge

In [ ]:
pub_df_concat = pd.concat([pub_df, pub_content_df], axis=1)

In [ ]:
pub_df_concat

## 본문 수집 - 방식 2

### url 구성에 필요한 정보 추출

In [ ]:
pub_url[0]

In [ ]:
apba_id = pub_url[0].split("'")[1]
print(apba_id)

In [ ]:
url = 'https://job.alio.go.kr/orginfoview.do?apba_id=' + apba_id
print(url)

In [ ]:
pub_real_url

In [ ]:
pub_contents = []
for u in pub_real_urls:
    driver.get(u)
    time.sleep(2)
    
    rows = driver.find_elements_by_xpath('//*[@id="txt"]/table/tbody/tr')
    alio_dict = {}
    for r in rows:
        header = r.find_element_by_tag_name('th').text
        content = r.find_element_by_tag_name('td').text
        alio_dict[header]=content
        
    pub_contents.append(alio_dict)

In [ ]:
pub_content_df = pd.DataFrame(pub_contents)
pub_content_df

### 정리 1

In [5]:
pub_contents = []

for u in pub_url:
    apba_id = u.split("'")[1]
    pub_real_url = 'https://job.alio.go.kr/orginfoview.do?apba_id=' + apba_id
    
    driver.get(pub_real_url)
    time.sleep(2)
    
    rows = driver.find_elements_by_xpath('//*[@id="txt"]/table/tbody/tr')
    
    alio_dict = {}
    for r in rows:
        header = r.find_element_by_tag_name('th').text
        content = r.find_element_by_tag_name('td').text
        alio_dict[header]=content
        
    pub_contents.append(alio_dict)

In [6]:
pub_content_df = pd.DataFrame(pub_contents)
pub_content_df

,기관명,기관소개,주무기관,설립근거,주요기능 및 역할,기관연혁,경영목표 및 전략,기관장,소재지
0,(사)남북교류협력지원협회,"정부 위탁업무 수행, 정책건의, 남북교류협력 관련 조사·연구 및 분석 등을 통한 남...",통일부,민법 32조,o 남북교류협력과 관련한 정부 위탁사업 수행\no 정부에 대한 남북교류협력 활성화 ...,o 2007.05.22 법인설립 인가(통일부)\no 2007.05.28「남북 경공업...,o 남북교류협력 활력 제고 및 성장동력 확보\n- 남북교류협력 원스톱 서비스 체계 ...,강영식,"중구 퇴계로 97, 601호"
1,(재)APEC기후센터,기후예측과 그 관련 연구를 통해 기후변화 및 변동의 영향에 효과적으로 대응할 수 있...,기상청,"민법 제32조, 공익법인의 설립·운영에 관한 법률","○ 기후변동 및 변화의 진단, 예측 관련 정보의 수집, 생산 및 유통\n○ 기후변동...",○ 2004.03 제4차 APEC 과학기술장관회의에서 설립 제안\n○ 2005.03...,○ 기후예측 전문성 강화 및 국내 기여 확대\n○ 기후예측 기술 개발 및 기후 감시...,권원태,해운대구 센텀7로 12
2,(재)국제원산지정보원,자유무역협정 및 원산지 관련 연구ㆍ조사ㆍ정보 보급 확대 등을 통하여 우리 기업의<b...,관세청,민법 제32조(비영리법인의 설립과 허가),1. 자유무역협정ㆍ원산지 정보의 수집ㆍ분석 수탁 사업\n2. 자유무역협정ㆍ원산지 정...,2009. 1 원산지정보 수집· 분석 전문기관 지정(관세청)\n2009. 2 「한국...,미션 : FTA 및 원산지 연구· 조사와 정보보급을 통해 기업경쟁력 강화와 국민경제...,박병진,성남시 분당구 야탑로 205번길8
3,(재)예술경영지원센터,"예술유통의 활성화와 예술기관의 경쟁력 강화를 종합적이고 체계적으로 지원함으로써, 예...",문화체육관광부,민법 제32조에 의한 재단법인,1. 예술 유통구조의 체계화 및 활성화 지원\n2. 예술기관·단체 운영 및 경영 관...,"○ 2006년 재단법인 예술경영지원센터 설립 (2006.1.12.), 제1기 이사회...",○ 비전 : 예술시장 활성화로 예술현장의 지속성장을 이끌어가는 기관\n○ 핵심가치 ...,김도일,"종로구 대학로 57(연건동) 홍익대학교 대학로캠퍼스 교육동 3층, 12층"
4,(재)우체국금융개발원,우체국금융 업무를 효과적으로 지원함으로써 우체국예금보험사업의 향상·발전에 기여\n홈...,과학기술정보통신부,"민법 제32조(비영리법인의 설립과 허가) 학술, 종교, 자선, 기예, 사교 기타 영...",ㅇ 국내외 금융시장에 대한 조사·연구\nㅇ 우체국금융 신서비스 도입 및 마케팅 방안...,1966.04. 재단법인 체신저축장려회 설립\n1976.12. 재단법인 체신장려회로...,◎ 미션 : 안전하고 편리한 금융서비스를 통해 국민의 삶의 질 향상에 기여한다\n\...,유대선,영등포구 경인로 841(영등포동4가)
5,(재)우체국물류지원단,안전ㆍ신속ㆍ정확한 우편물류서비스를 효율적으로 제공함으로써 국민의 편익 <br />증...,과학기술정보통신부,민법 제32조,"'우체국물류지원단'은 전국 우편물 운송, 배달, 분류, 국제우편물 포워딩 등 종합우...",□ 1980.08.26 법인설립 (재)체신복지회\n□ 1981.03.01 수도권 우...,□ (기관미션) 안전ㆍ 신속ㆍ 정확한 우편물류서비스를 효율적으로 제공함으로써\n국민...,천장수,광진구 자양로 76
6,(재)우체국시설관리단,우정사업조직에 속한 부동산의 효율적 관리ㆍ운영으로 우정자산의 적극적 활용 및 가치향...,과학기술정보통신부,민법 제32조(비영리법인의 설립과 허가),o 우정재산의 사용ㆍ수익 허가 및 대부업무 지원사업\no 우정사업 부동산 시설 유지...,o 2001. 11. 23. 법인 설립\no 2006. 12. 28. 우체국사 관리...,⊙ 미션 : 전문적이고 효율적인 우정시설 및 자산관리를 통해 공공의 복리 증진에 기...,김성칠,"광진구 강벽역로 2(구의동, 우체국시설관리단)"
7,(재)일제강제동원피해자지원재단,"일제강제동원 피해·희생자 및 유족 등에 대한 복지지원사업, 추념사업 및 강제동원 피...",행정안전부,"- 설립근거「민법」제32조(비영리 법인의 설립과 허가),「공익법인의 설립·운영에 관...",- 일제 강제동원 희생자에 대한 추도순례 등 위령사업- 해외 추도공간 조성 및 일제...,- 2014. 6. 2. / 3. 재단 설립허가/등기- 2015. 1. 29. 기타...,"- 미션: 과거의 기억, 현재의 통합, 미래의 평화- 비젼: 국민과 함께 평화와 인...",김용덕,"종로구 종로1길 42, 603호(수송동, 이마빌딩)"
8,(재)축산환경관리원,친환경적인 가축사육환경 조성 및 가축분뇨의 자원화를 효율적으로 수행함으로써<br /...,농림축산식품부,「가축분뇨의 관리 및 이용에 관한 법률」제38조의2(축산환경관리원의 설립·운영),"o 배출시설 또는 처리시설 컨설팅, 지도 및 교육, 퇴비·액비의 품질관리\no 가축...","o 민간관리기구 도입검토('10.7월, 국무총리실 주관 T/F팀)o '가축분뇨의 효...",o (비전) 국민에게 신뢰받는 세계 최고의 축산환경개선 선도기관\no (미션) 지속...,이영희,"한누리대로 219, 7층(나성동, 한림프라자)"
9,(재)한국우편사업진흥원,우편서비스의 질적 향상 및 우정문화를 창달함으로써 우정사업 발전과 국민문화 생활에 ...,과학기술정보통신부,민법 제32조(비영리법인의 설립과 허가),1. 문화우편상품과 우표류 보급 및 콘텐츠를 통한 우정문화 확산\n2. 우정 및 그...,ㅇ 1930. 12. 03 우편소청사협회 설립\nㅇ 1939. 11. 06 조선체신...,ㅇ 미션: 우정문화의 가치혁신과 선진 우편서비스의 제공으로 국민의 행복과 공익실현에...,민재석,영등포구 영중로 83


### 정리 2

In [9]:
import csv

file_name = "공공기관정보(job-alio).csv"
f = open(file_name, 'w', encoding='utf-8-sig', newline='')
writer = csv.writer(f)

for u in pub_url:
    apba_id = u.split("'")[1]
    pub_real_url = 'https://job.alio.go.kr/orginfoview.do?apba_id=' + apba_id
    
    driver.get(pub_real_url)
    time.sleep(2)
    
    rows = driver.find_elements_by_xpath('//*[@id="txt"]/table/tbody/tr')
    
    content = []
    for r in rows:
        content.append(r.find_element_by_tag_name('td').text)
    writer.writerow(content)    
f.close()

## Headless Chrome

### User Agents

웹사이트에 접속할 때 어느 브라우저를 쓰는지, OS를 쓰는지, 컴퓨터로 접속하는지, 스마트폰으로 접속하는지 등 정보를 서버가 알 수 있음  
사람이 접속하는 것이 아니라 컴퓨터가 접속하는 경우 사이트에서는 접속을 차단할 수 있음  

https://www.whatismybrowser.com/detect/what-is-my-user-agent

In [ ]:
headless_options = webdriver.ChromeOptions()
headless_options.headless = True
headless_options.add_argument('window-size=1920x1080')  
headless_options.add_argument('User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36')  
headless_options.add_argument('lang=ko_KR')  
headless_options.add_argument('disable-gpu')  

In [ ]:
from selenium import webdriver
import pandas as pd
import time

headless_driver = webdriver.Chrome('./chromedriver', options=headless_options)
url = 'https://job.alio.go.kr/orginfo.do'
headless_driver.get(url)

In [ ]:
headless_driver.get_screenshot_as_file("./img/Job-Alio_first.png")

screenshot 

![screenshot_1](./img/Job-Alio_first.png)

In [ ]:
checkbox1 = headless_driver.find_element_by_xpath('//*[@id="so01"]')
checkbox1.click()

In [ ]:
headless_driver.get_screenshot_as_file("./img/Job-Alio_checkbox.png")

screenshot2 

![screenshot_2](./img/Job-Alio_checkbox.png)

In [ ]:
pub_contents = []

for u in pub_url:
    apba_id = u.split("'")[1]
    pub_real_url = 'https://job.alio.go.kr/orginfoview.do?apba_id=' + apba_id

    headless_driver.get(pub_real_url)
    time.sleep(2)
    print(f'processing {apba_id}')
    
    rows = headless_driver.find_elements_by_xpath('//*[@id="txt"]/table/tbody/tr')
    alio_dict = {}
    for r in rows:
        header = r.find_element_by_tag_name('th').text
        content = r.find_element_by_tag_name('td').text
        alio_dict[header]=content
        
    pub_contents.append(alio_dict)

In [ ]:
pub_content_df1 = pd.DataFrame(pub_contents)
pub_content_df1

In [ ]:
headless_driver.quit()

## Requests & BeautifulSoup

In [ ]:
import requests
from bs4 import BeautifulSoup

pub_contents = []
for u in pub_url:
    apba_id = u.split("'")[1]
    pub_real_url = 'https://job.alio.go.kr/orginfoview.do?apba_id=' + apba_id

    res = requests.get(pub_real_url)
    time.sleep(2)
    print(f'processing {apba_id}')
    
    soup = BeautifulSoup(res.text, 'html.parser')
    rows = soup.select('#txt > table > tbody > tr')
    
    alio_dict = {}
    for r in rows:
        header = r.select_one('th').get_text()
        content = r.select_one('td').get_text()
        alio_dict[header]=content
        
    pub_contents.append(alio_dict)

In [ ]:
pub_content_df_request = pd.DataFrame(pub_contents)
pub_content_df_request

### HTTP response code

HTTP라는 프로토콜 규격에 따라서 응답 데이터에 응답 코드를 함께 받음  
https://www.w3.org/Protocols/rfc2616/rfc2616-sec10.html

응답코드가 200인 경우는 정상 응답

In [ ]:
res = requests.get('https://job.alio.go.kr/orginfoview.do?apba_id=C0847')
print(res.status_code)

In [ ]:
import requests
from bs4 import BeautifulSoup

pub_contents = []
for u in pub_url:
    apba_id = u.split("'")[1]
    pub_real_url = 'https://job.alio.go.kr/orginfoview.do?apba_id=' + apba_id

    res = requests.get(pub_real_url)
    time.sleep(2)
    
    if res.status_code == 200:
        print(f'processing {apba_id}')

        soup = BeautifulSoup(res.text, 'html.parser')
        rows = soup.select('#txt > table > tbody > tr')
        alio_dict = {}
        
        for r in rows:
            header = r.select_one('th').get_text()
            content = r.select_one('td').get_text()
            alio_dict[header]=content

        pub_contents.append(alio_dict)
    else:
        print(f'{apba_id}: {res.status_code}')

### User Agents

In [ ]:
import requests

res = requests.get('https://naver.com')
res.request.headers

In [8]:
import requests

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}

res = requests.get('https://naver.com', headers=headers)
res.request.headers

{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

# 동적 웹페이지 구조

![network image2](./img/network_2.jpg)

![network image3](./img/network_3.jpg)

# 예제 3: 서울복지포털

https://wis.seoul.go.kr/hope/customizedSearch.do

In [ ]:
from selenium import webdriver
import pandas as pd
import time

In [ ]:
driver = webdriver.Chrome('./chromedriver.exe')
url = 'https://wis.seoul.go.kr/hope/customizedSearch.do'
driver.get(url)

## 방식 1

In [ ]:
# //*[@id="content"]/div[1]/div[2]/dl[1]/dd/div/div/table
# //*[@id="content"]/div[1]/div[2]/dl[2]/dd/div/div/table

center_info = driver.find_elements_by_tag_name('table')
print(len(center_info))
for c in center_info:
    print(c.text)

## 방식 2

In [ ]:
center_list = []
centers = driver.find_elements_by_xpath('//*[@id="content"]/div[1]/div[2]/dl')
for center in centers:
    drop_btn = center.find_element_by_xpath('dt/button')
    drop_btn.click()
    time.sleep(2)
    center_info = {}
    ths = center.find_elements_by_tag_name('th')
    tds = center.find_elements_by_tag_name('td')
    for n in range(len(ths)):
        center_info[ths[n].text]=tds[n].text
    center_list.append(center_info)

In [ ]:
center_df = pd.DataFrame(center_list)
center_df

## 방식 3

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(driver.page_source, 'html.parser')
centers = soup.select('#content > div.sub_content > div.board_t1.no_out > dl')
center_list = []
for center in centers:
    center_info = {}
    ths = center.select('th')
    tds = center.select('td')
    for n in range(len(ths)):
        center_info[ths[n].get_text()]=tds[n].get_text()
    center_list.append(center_info)

In [ ]:
center_df = pd.DataFrame(center_list)
center_df

## Pagination

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
driver = webdriver.Chrome('./chromedriver.exe')
url = 'https://wis.seoul.go.kr/hope/customizedSearch.do'
driver.get(url)

In [ ]:
current_page_n = 1
center_list = []
while current_page_n < 3:
    driver.execute_script(f'pagedIng({current_page_n});')
    print(f'processing page {current_page_n}')
    time.sleep(3)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    centers = soup.select('#content > div.sub_content > div.board_t1.no_out > dl')
    
    for center in centers:
        center_info = {}
        ths = center.select('th')
        tds = center.select('td')
        for n in range(len(ths)):
            center_info[ths[n].get_text()]=tds[n].get_text().strip(' \t')
        center_list.append(center_info)
        
    current_page_n += 1

In [ ]:
center_df = pd.DataFrame(center_list)
center_df

# 비교: 나라일터

https://www.gojobs.go.kr/frameMenu.do?url=apmList.do%3FsearchJobsecode%3D050%26searchEmpmnsecode%3De05&menuNo=47&mngrMenuYn=N&message=